# Imports

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

2024-06-05 13:03:10.400216: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-05 13:03:10.400442: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-05 13:03:10.538766: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Read Data

In [2]:
X_train = pd.read_csv("/kaggle/input/ibm-classification-feature-selection/X_train_normalized.csv", index_col=[0])
X_test = pd.read_csv("/kaggle/input/ibm-classification-feature-selection/X_test_normalized.csv", index_col=[0])
y_train = pd.read_csv("/kaggle/input/ibm-classification-feature-selection/y_train.csv", index_col=[0])
y_test = pd.read_csv("/kaggle/input/ibm-classification-feature-selection/y_test.csv", index_col=[0]).reset_index(drop=True)

In [3]:
X_train.head(5)

,Symbol,Volume,Sector_Basic Materials,Sector_Communication Services,Sector_Consumer Cyclical,Sector_Consumer Defensive,Sector_Energy,Sector_Financial Services,Sector_Healthcare,Sector_Industrials,...,Adj Close 14-Day std Differenced,Adj Close 26-Day std Differenced,Adj Close 7-Day Upper Bollinger Band Differenced,Distance to Adj Close 7-Day Upper Bollinger Band Differenced,True Range Differenced,True Range 26-Day SMA Differenced,Adj Close Differenced,High Differenced,Low Differenced,Open Differenced
0,-1.439324,-0.326871,-0.21136,-0.21136,-0.356345,-0.279992,-0.222173,-0.394553,-0.387284,-0.412467,...,0.041515,0.037984,-0.002604,-0.016884,-0.066039,0.104504,-0.020330,0.008943,0.015877,0.062622
1,-1.069061,-0.344648,-0.21136,-0.21136,-0.356345,-0.279992,-0.222173,-0.394553,2.582087,-0.412467,...,-0.410893,-0.015098,0.023690,-0.111938,-0.141182,-0.090630,-0.107768,0.032258,0.030478,0.096788
2,1.641263,-0.295005,-0.21136,-0.21136,-0.356345,-0.279992,-0.222173,-0.394553,-0.387284,-0.412467,...,0.081853,0.002250,0.188618,0.000862,-0.002066,0.105746,0.125379,0.102205,0.158246,0.137784
3,-0.343346,0.285874,-0.21136,-0.21136,-0.356345,-0.279992,-0.222173,-0.394553,2.582087,-0.412467,...,0.162828,-0.007191,0.102354,0.062595,0.043600,0.091515,0.136523,0.094434,0.004925,0.076288
4,1.256190,-0.328472,-0.21136,-0.21136,-0.356345,-0.279992,-0.222173,2.534516,-0.387284,-0.412467,...,1.195841,-0.202535,1.265493,-0.284640,-0.679202,-1.099516,0.521058,0.755054,0.866425,0.817676


In [4]:
X_test.head(5)

,Symbol,Volume,Sector_Basic Materials,Sector_Communication Services,Sector_Consumer Cyclical,Sector_Consumer Defensive,Sector_Energy,Sector_Financial Services,Sector_Healthcare,Sector_Industrials,...,Adj Close 14-Day std Differenced,Adj Close 26-Day std Differenced,Adj Close 7-Day Upper Bollinger Band Differenced,Distance to Adj Close 7-Day Upper Bollinger Band Differenced,True Range Differenced,True Range 26-Day SMA Differenced,Adj Close Differenced,High Differenced,Low Differenced,Open Differenced
0,-1.054250,-0.283809,-0.21136,-0.21136,-0.356345,-0.279992,-0.222173,-0.394553,-0.387284,2.424439,...,0.244619,-0.054833,0.193442,-0.988178,-0.460024,-0.458573,-0.961706,-0.123181,-0.779922,0.216366
1,0.449017,-0.359859,-0.21136,-0.21136,-0.356345,-0.279992,-0.222173,2.534516,-0.387284,-0.412467,...,-1.623606,-1.598235,-0.116220,-0.549677,-0.673831,-0.614296,-0.682605,0.832754,-0.079040,1.432648
2,1.419105,-0.198809,-0.21136,-0.21136,-0.356345,-0.279992,-0.222173,-0.394553,-0.387284,-0.412467,...,-4.366291,-2.195166,0.093043,-4.112427,3.947181,1.902798,-4.471950,1.435083,-4.328154,1.015831
3,-1.261598,-0.272044,-0.21136,-0.21136,-0.356345,-0.279992,-0.222173,-0.394553,-0.387284,-0.412467,...,5.267571,4.461425,4.610485,-12.390394,14.154228,10.022441,-10.617055,-3.111471,-14.261016,-12.872375
4,-0.128593,-0.255868,-0.21136,-0.21136,2.806268,-0.279992,-0.222173,-0.394553,-0.387284,-0.412467,...,0.039264,0.059860,0.030323,-0.411204,0.082851,0.121352,-0.433286,-0.263072,-0.294412,0.025041


In [5]:
y_train.head(5)

,1-week Forward Return Sign
0,0.0
1,0.0
2,0.0
3,0.0
4,1.0


In [6]:
y_test.head(5)

,1-week Forward Return Sign
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0


# Data Prep

LSTMs requires the data to be structured differently to most ML models. The idea is to look at a 'window' of data (e.g. last 5 timesteps) and use it to predict the target at the next timestep. We can roll the window to look at the whole dataset but only looking at one window at a time. 

To do this, we convert our dataframe to a 3D array X and a 1D array y. The array X has shape (n_samples, window_size, num_features) and the array y simply contains the target for each sample. The structure of the array X is shown below for a window size of 2 (LHS) and the array y is shown on the RHS. Note we have filtered to one stock but in reality, we have all the stocks and the rows appear ordered by time, not stock, because otherwise we would encounter look-ahead bias when splitting the dataset into training and validation (because we would be using for example stock 2's data at time T+1 in the training set to predict stock 1's data at time T in the validation set)

[Stock1Feature1(1), Stock1Feature2(1), ....., Stock1Featuren(1)], [Stock1Feature1(0), Stock1Feature2(0), ....., Stock1Featuren(0)]                                                                             [Stock1Target(2)]

[Stock1Feature1(2), Stock1Feature2(2), ....., Stock1Featuren(2)], [Stock1Feature1(1), Stock1Feature2(1), ....., Stock1Featuren(1)]                                                                             [Stock1Target(3)]

[Stock1Feature1(3), Stock1Feature2(3), ....., Stock1Featuren(3)], [Stock1Feature1(2), Stock1Feature2(2), ....., Stock1Featuren(2)]                                                                             [Stock1Target(4)]

....





In [7]:
def prep_data_for_lstm(df, window):
    """function to put data in the correct format for LSTM. It assumes that the 'Target' is the last column of the dataframe."""
    
    # Convert dataframe to numpy
    df_as_np = df.to_numpy()

    # Get unique stock ids
    stock_ids = df['Symbol'].unique()

    # Array to store X and y
    X = []
    y = []

    # Loop through each index in the dataframe and get X and y
    for i in range(len(df_as_np)-window):
        # Get values in window and wrap each one in a list
        row = df_as_np[i: i+window]
        X.append(row)

        # Get the label (first value after the window)
        label = df_as_np[i + window][-1]
        y.append(label)
    
    return np.array(X), np.array(y)

In [8]:
# Window size - use last n samples to predict next sample
window = 2

# Convert training data ready for LSTM
df = pd.concat([X_train, y_train], axis=1)
X, y = prep_data_for_lstm(
    df=df,
    window=window
)
print(X)
print(y)
print(X.shape)
print(y.shape)

[[[-1.43932383 -0.32687053 -0.21135994 ...  0.01587707  0.06262152
    0.        ]
  [-1.06906097 -0.34464772 -0.21135994 ...  0.03047779  0.09678764
    0.        ]]

 [[-1.06906097 -0.34464772 -0.21135994 ...  0.03047779  0.09678764
    0.        ]
  [ 1.64126314 -0.29500516 -0.21135994 ...  0.15824566  0.13778437
    0.        ]]

 [[ 1.64126314 -0.29500516 -0.21135994 ...  0.15824566  0.13778437
    0.        ]
  [-0.34334577  0.28587356 -0.21135994 ...  0.00492478  0.07628797
    0.        ]]

 ...

 [[ 0.81187434 -0.36048932 -0.21135994 ...  0.77152081  0.38719505
    0.        ]
  [ 0.197238   -0.29831467 -0.21135994 ... -0.01332578 -0.0501247
    0.        ]]

 [[ 0.197238   -0.29831467 -0.21135994 ... -0.01332578 -0.0501247
    0.        ]
  [-0.9431716  -0.34905028 -0.21135994 ...  0.03048058 -0.31661518
    1.        ]]

 [[-0.9431716  -0.34905028 -0.21135994 ...  0.03048058 -0.31661518
    1.        ]
  [-1.57261846 -0.35644353 -0.21135994 ...  0.12539019 -0.21753279
    0.

In [9]:
# Split training data into a training set and validation set with small gap
import math
split = math.floor(0.8 * X.shape[0])
X_train_lstm, y_train_lstm = X[:split], y[:split]
X_val_lstm, y_val_lstm = X[split + 4000:], y[split + 4000:]
print(X_train_lstm.shape, y_train_lstm.shape)
print(X_val_lstm.shape, y_val_lstm.shape)

(505860, 2, 123) (505860,)
(122465, 2, 123) (122465,)


# Fit LSTM
An explanation of the model's layers can be seen in the comments in the code.

In [10]:
from keras.models import Sequential
from keras.layers import InputLayer, Dense, LSTM
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.metrics import AUC
from keras.optimizers import Adam
from keras.regularizers import l2

In [11]:
# Initialize sequential model in keras. This is a model that expects a linear stack of layers, meaning that each layer has one input tensor and one output tensor. 
model = Sequential()

# Add input layer. Each input tensor will be an element of X_train_lstm so the tensor will have shape (window, n_features)
model.add(InputLayer((window, X_train_lstm.shape[2]))) # Window size and number of variables used to predict

# Add LSTM layer. This is a recurrent layer that can learn long-term dependencies in sequences.
model.add(
    LSTM(
        units=64,
        dropout=0.2, # Randomly set fraction of non-recurrent LSTM units to 0 at each training time step to reduce relianc on units and prevent overfitting
        recurrent_dropout=0.2 # Randomly set fraction of recurrent LSTM units to 0 at each training time step to reduce relianc on units and prevent overfitting
    )
)

# Add hidden layer to take LSTM layer output and transform it to a higher-level feayire space.
model.add(
    Dense(
        units=8, 
        activation='relu',  # relu activation function adds some non-linearity without being computatinally expensive
        kernel_regularizer=l2(0.01) # Add l2 regularization on layer to reduce magnitude of weights reducing overfitting
    )
)

# Add output layer with sigmoid activation function to output a value between 0 and 1 interpreted as probability of postiivre class
model.add(Dense(1, 'sigmoid'))

# Print summary of model
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 64)             │        48,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 8)              │           520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 48,657 (190.07 KB)

 Trainable params: 48,657 (190.07 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
# Checpoint for saving model
cp1 = ModelCheckpoint(
    'lstm_model.keras',
    save_best_only=True
)

# Early stopping rounds checkpint
cp2 =  EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [13]:
# Compile model
model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(learning_rate=0.0001),
    metrics=[AUC(name='auc')]
)

In [14]:
# Adjust class weights inversely proportional to class frequencies
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight(
    class_weight='balanced', 
    classes=np.unique(y_train_lstm),
    y=y_train_lstm
)
class_weight_dict = dict(enumerate(class_weights))

# Fit model by measiromg AUC against validation set.
model.fit(
    X_train_lstm,
    y_train_lstm,
    validation_data=(X_val_lstm, y_val_lstm),
    epochs=100, # Number of times to run through the data
    callbacks=[cp1, cp2], # At end of every epoch, save the model if loss was lower than before. Also EarlyStoppingRounds
    class_weight=class_weight_dict # Handle imbalance of classes
)

Epoch 1/100
15809/15809 ━━━━━━━━━━━━━━━━━━━━ 74s 4ms/step - auc: 0.6048 - loss: 0.7210 - val_auc: 0.5913 - val_loss: 0.6767
Epoch 2/100
15809/15809 ━━━━━━━━━━━━━━━━━━━━ 69s 4ms/step - auc: 0.6693 - loss: 0.6503 - val_auc: 0.5719 - val_loss: 0.6834
Epoch 3/100
15809/15809 ━━━━━━━━━━━━━━━━━━━━ 67s 4ms/step - auc: 0.6825 - loss: 0.6415 - val_auc: 0.5709 - val_loss: 0.6826
Epoch 4/100
15809/15809 ━━━━━━━━━━━━━━━━━━━━ 68s 4ms/step - auc: 0.6904 - loss: 0.6361 - val_auc: 0.5547 - val_loss: 0.6889
Epoch 5/100
15809/15809 ━━━━━━━━━━━━━━━━━━━━ 71s 5ms/step - auc: 0.6969 - loss: 0.6315 - val_auc: 0.5602 - val_loss: 0.6886
Epoch 6/100
15809/15809 ━━━━━━━━━━━━━━━━━━━━ 72s 5ms/step - auc: 0.6991 - loss: 0.6297 - val_auc: 0.5488 - val_loss: 0.6919
